In [1]:
%pip install pyspark
%pip install delta-spark

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
path_csv = '../../dados/rd/dbc/csv'

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr
from delta.tables import DeltaTable
import shutil

In [4]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

builder = SparkSession.builder.appName("DeltaLakeApp") \
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
  .config("spark.jars.packages","io.delta:delta-core_2.12:2.0.0")

spark =  configure_spark_with_delta_pip(builder).getOrCreate()

24/03/27 14:52:08 WARN Utils: Your hostname, codespaces-d1cb62 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/03/27 14:52:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/python/3.10.13/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/codespace/.ivy2/cache
The jars for the packages stored in: /home/codespace/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-87dd69c8-8549-43da-b33f-ed4c7063814f;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 245ms :: artifacts dl 18ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3  

In [6]:
df = spark.read.csv(path_csv, sep=',', header=True)

In [8]:
df.show(2)

#df.count()
#df.columns

+------+--------+--------+-----+--------------+-------------+-----+--------+---------+--------+----+----------+----------+----------+----------+---------+----------+----------+----------+----------+---------+----------+----------+----------+------+------+--------+------+---------+--------+----------+----------+----------+----------+----------+-------+-------+------+--------+--------+----------+----------+--------+--------+-------+------+-------+--------+---------+---------+-----+---------+-----+--------+--------+-------+---------+-------+--------+----------+------+---------+----------+----------+---------+------------+--------+------+-----+--------+----------+---------+---------------+---------+-------+---------+--------+--------+---------+-------+------+-------+-----+--------+-----+---------+--------------------+--------+--------+----------+----------+----------+----------+----------+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--

In [22]:
#create a database
%cd '/workspaces/teo-me-why-datasus/tmp'
spark.sql("create database if not exists bronze")
spark.sql("use bronze")

/home/codespace/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/workspaces/teo-me-why-datasus/tmp


DataFrame[]

In [23]:
df.write.mode("overwrite").format("delta").saveAsTable("datasus")
'''
deste modo o spark gera uma tabela e preenche com os dados; todos os novos dados
são inserido no final (append)
quando um dados for atualizado, não deve fazer append e, sim, update
atualização baseada em chave primária
PROBLEMA: BD datasus não tem chave primária (PK) definida
'''

'\ndeste modo o spark gera uma tabela e preenche com os dados; todos os novos dados\nsão inserido no final (append)\nquando um dados for atualizado, não deve fazer append e, sim, update\natualização baseada em chave primária\nPROBLEMA: BD datasus não tem chave primária (PK) definida\n'

In [24]:
df_delta = spark.read.table("bronze.datasus")
df_delta.show(2)

+------+--------+--------+-----+--------------+-------------+-----+--------+---------+--------+----+----------+----------+----------+----------+---------+----------+----------+----------+----------+---------+----------+----------+----------+------+------+--------+------+---------+--------+----------+----------+----------+----------+----------+-------+-------+------+--------+--------+----------+----------+--------+--------+-------+------+-------+--------+---------+---------+-----+---------+-----+--------+--------+-------+---------+-------+--------+----------+------+---------+----------+----------+---------+------------+--------+------+-----+--------+----------+---------+---------------+---------+-------+--------------+--------+--------+---------+-------+------+-------+-----+--------+-----+---------+--------------------+--------+--------+----------+----------+----------+----------+----------+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------

In [36]:
#%pip install fastparquet pyarrow

Note: you may need to restart the kernel to use updated packages.


In [39]:
df_delta.write.parquet('df_delta.parquet')

In [40]:
import os
file_name = "/workspaces/teo-me-why-datasus/02.bronze/datasus/df_delta.parquet"

file_stats = os.stat(file_name)

print(file_stats)
print(f'File Size in Bytes is {file_stats.st_size}')
print(f'File Size in MegaBytes is {file_stats.st_size / (1024 * 1024)}')

os.stat_result(st_mode=16877, st_ino=1458481, st_dev=1795, st_nlink=2, st_uid=1000, st_gid=1000, st_size=4096, st_atime=1711553236, st_mtime=1711553236, st_ctime=1711553236)
File Size in Bytes is 4096
File Size in MegaBytes is 0.00390625


Comentário sobre a chave primária:
* N_AIH: número da internação
* //cada internação pode ser estendida e pode receber um novo número
* DT_SAIDA
* IDENT



In [ ]:
'''
verificação se chave composta (N_AIH, DT_SAIDA, IDENT) é PRIMÁRIA
'''

#spark.sql(f'''SELECT
#          COUNT(DISTINCT N_AIH, DT_SAIDA, IDENT),
#          COUNT(*)
#          FROM bronze.datasus''').show()

In [ ]:
schema = df_delta.schema
schema.jsonValue()

In [ ]:
df_stream = spark.readStream\
.format("csv")\
.schema(schema)\
.load(path_csv)

'setar limie 100 arquivos por trigger'

In [ ]:
checkpoint_folder = '/drive/MyDrive/analise_dados/dados/rd/checkpoint'
os.makedirs(checkpoint_folder)

df_stream.writeStream.\
  .option("checkpointLocation", checkpoint_folder)\
  .foreachBatch()
  .start()

#upsert - github Anne

In [ ]:
spark.sql("select * from bronze.datasus").show()